<a href="https://colab.research.google.com/github/Ej0007/CCDEPLRL_EXERCISES_COM211/blob/main/Guevara_Exercise5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 5

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import numpy as np

In [ ]:
tokenizer = Tokenizer()

data = open('sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

1. Create the LSTM model

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(40, return_sequences=True))) #LSTM Layer #1
model.add(Dropout(0.2)) #Dropout layer
model.add(Bidirectional(LSTM(40))) #LSTM Layer #2
#model.add(Dense()) #Regularizer
model.add(Dense(total_words, activation='softmax')) #Dense
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 100)           337500    
                                                                 
 bidirectional_2 (Bidirecti  (None, 10, 80)            45120     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 10, 80)            0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 80)                38720     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 3375)              273375    
                                                                 
Total params: 694715 (2.65 MB)
Trainable params: 69471

In [ ]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 18s 21ms/step - loss: 6.9194 - accuracy: 0.0234
Epoch 2/100
484/484 [==============================] - 6s 13ms/step - loss: 6.4885 - accuracy: 0.0295
Epoch 3/100
484/484 [==============================] - 6s 12ms/step - loss: 6.3438 - accuracy: 0.0369
Epoch 4/100
484/484 [==============================] - 6s 12ms/step - loss: 6.2278 - accuracy: 0.0375
Epoch 5/100
484/484 [==============================] - 6s 12ms/step - loss: 6.1203 - accuracy: 0.0405
Epoch 6/100
484/484 [==============================] - 6s 12ms/step - loss: 6.0039 - accuracy: 0.0446
Epoch 7/100
484/484 [==============================] - 5s 11ms/step - loss: 5.8827 - accuracy: 0.0532
Epoch 8/100
484/484 [==============================] - 7s 14ms/step - loss: 5.7616 - accuracy: 0.0577
Epoch 9/100
484/484 [==============================] - 5s 11ms/step - loss: 5.6399 - accuracy: 0.0623
Epoch 10/100
484/484 [==============================] - 6s 13ms/step - loss: 5.52

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)